In [13]:
# required libraries
import re
import spacy
import string
import textacy.ke
import numpy as np
import pandas as pd
import datetime as dt
import streamlit as st
import matplotlib.pyplot as plt

%matplotlib inline

from PIL import Image
from spacy import displacy
from tabulate import tabulate
from string import punctuation
from collections import Counter
from nltk.corpus import stopwords
from spacy.matcher import Matcher
from transformers import pipeline
from nltk.stem import WordNetLemmatizer
from wordcloud import WordCloud, STOPWORDS
from gensim.models import Word2Vec, KeyedVectors
from nltk.tokenize import sent_tokenize, word_tokenize

In [2]:
classifier = pipeline('sentiment-analysis')

In [14]:
# Load the local files
rleaves = pd.read_csv('rleaves.csv', encoding='utf-8')

# Cleaning up the corpus
sp = spacy.load('en_core_web_sm')

# a custom function for complete text cleanup
def cleanup1(text):
    text = text.lower() # lowers the corpus
    text = re.sub('http\S+', ' ', str(text)) # removes any url
    text = re.sub('n\'t\s', ' not ', str(text))
    text = re.sub('-(?<!\d)', ' ', str(text)) # removing hyphens from numbers
    text = sp(text) # apply spacy model
    text = [w.text for w in text if not w.is_stop] # tokenize and remove stop words
    text = sp(' '.join(text)) # join words and apply spacy model again 
    text = [w.lemma_ for w in text] # lemmatizes the words
    stopwords_extra = ['im', 'na', 'u', 'ill', '10184285', '179180', 'as', 'oh', 'av', 'wo', 'nt', 'p', 'm', 'ta', '10000', '6000']
    text = [word for word in text if not word in stopwords_extra] # remove additional unnecessary words
    text = ' '.join(text)  # join the words back together  
    text = text.translate(str.maketrans('', '', string.punctuation)) # removes all punctuation
    text = re.sub('[^\w\s,]', ' ', str(text)) # removes emoticon and other non characters
    text = re.sub('x200b', ' ', str(text)) # removing zero-width space characters
    text = re.sub(' cannabi ', ' cannabis ', str(text))
    return ' '.join([token for token in text.split()]) # removes trailing whitespaces

# a custom function to change UTC time and split days and hours
def change_time(utc):
    day = dt.datetime.fromtimestamp(utc).strftime('%A')
    hour = dt.datetime.fromtimestamp(utc).strftime('%I %p')
    return pd.Series([day, hour])

# apply preprocessing function
rleaves[['day', 'hour']] = rleaves['time'].apply(change_time)
rleaves['raw'] = pd.DataFrame(rleaves['raw'].apply(cleanup1))

In [19]:
print(classifier(rleaves['raw'][1]))

[{'label': 'POSITIVE', 'score': 0.9896752834320068}]


In [22]:
rleaves['raw'][0]

'note encouragement meditation meditate nearly 20 year see people express frustration mediation r leave word u tommybahami123 kind tough sense distract attempt clear mind u jonny070389 give meditation mind go crazy tangent meditate year credit meditation make life easy place live think pass thinking thank leaver inspiration write say give meditation mind go crazy tangent like say give work heart rate go start breathe hard mind go crazy tangent exactly meditate experience beginning learn guide tangent peace exactly learn everybody mind go crazy tangent mediation learn learn long time suggestion begin meditation attitude go experience go say going make job chore skill feel develop little ambitious starting suggestion sit quietly comfortable position darkish pitch black room set timer minute tell voice head yammer away minute go listen sit quietly listen try guide try mental note say try tell remember later sit quietly listen alarm go day try process hear think wow obsessed work let minut

In [23]:
summarizer = pipeline("summarization")
article = '''note encouragement meditation meditate nearly 20 year see people express frustration mediation r leave word u tommybahami123 kind tough sense distract attempt clear mind u jonny070389 give meditation mind go crazy tangent meditate year credit meditation make life easy place live think pass thinking thank leaver inspiration write say give meditation mind go crazy tangent like say give work heart rate go start breathe hard mind go crazy tangent exactly meditate experience beginning learn guide tangent peace exactly learn everybody mind go crazy tangent mediation learn learn long time suggestion begin meditation attitude go experience go say going make job chore skill feel develop little ambitious starting suggestion sit quietly comfortable position darkish pitch black room set timer minute tell voice head yammer away minute go listen sit quietly listen try guide try mental note say try tell remember later sit quietly listen alarm go day try process hear think wow obsessed work let minute session day week second week actively choose somthe help set timer choose simple repetitive concentrate people focus breathing actual sensation air moving nose throat people count slowly quietly head choose make sense peaceful repetitive minute let thought rest repetitive idea inhale exhale inhale exhale choose nearly immediately lose control chatty thought intrude happen simply think quietly thought want peaceful repetition return choose lose focus happen time session quietly guide repetitive place teach frustrated time happen note remark happen return peaceful repetition task work meditation note thought stray return peaceful repetition meditate nearly 20 year practice inner thought intrude few few time experience return place peace suspect skill learn meditating begin extend outside meditation session learn aware inner dialog capable noting say separate keep take automatic influence thought action hope sense feel free ama clear'''

2020-12-31 13:56:02.788 INFO    filelock: Lock 140086265059264 acquired on /home/saayed/.cache/huggingface/transformers/adac95cf641be69365b3dd7fe00d4114b3c7c77fb0572931db31a92d4995053b.9307b6cec4435559ec6e79d5a210a334b17706465329e138f335649d14f27e78.lock


2020-12-31 13:56:03.085 INFO    filelock: Lock 140086265059264 released on /home/saayed/.cache/huggingface/transformers/adac95cf641be69365b3dd7fe00d4114b3c7c77fb0572931db31a92d4995053b.9307b6cec4435559ec6e79d5a210a334b17706465329e138f335649d14f27e78.lock


2020-12-31 13:56:03.337 INFO    filelock: Lock 140054398212416 acquired on /home/saayed/.cache/huggingface/transformers/b336fa0b874ea92e3e22f07a7e6f8fa9da01221759c33abeb2679d6d98fe7755.585965cf7e82e4536033cd21d76c486af3d6b1c2a34b3a847840d4e7fe9d8844.lock


2020-12-31 13:56:32.764 INFO    filelock: Lock 140054398212416 released on /home/saayed/.cache/huggingface/transformers/b336fa0b874ea92e3e22f07a7e6f8fa9da01221759c33abeb2679d6d98fe7755.585965cf7e82e4536033cd21d76c486af3d6b1c2a34b3a847840d4e7fe9d8844.lock


2020-12-31 13:56:44.845 INFO    filelock: Lock 140054398694400 acquired on /home/saayed/.cache/huggingface/transformers/9951e68693b9a7c583ae677e9cb53c02715d9bd0311a78706401372653cdea0a.647b4548b6d9ea817e82e7a9231a320231a1c9ea24053cc9e758f3fe68216f05.lock


2020-12-31 13:56:45.377 INFO    filelock: Lock 140054398694400 released on /home/saayed/.cache/huggingface/transformers/9951e68693b9a7c583ae677e9cb53c02715d9bd0311a78706401372653cdea0a.647b4548b6d9ea817e82e7a9231a320231a1c9ea24053cc9e758f3fe68216f05.lock


2020-12-31 13:56:45.665 INFO    filelock: Lock 140054397914368 acquired on /home/saayed/.cache/huggingface/transformers/7588c8d398d659b230a038240cc023f67b6848117d2999f06ab625af7bfc7ec1.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b.lock


2020-12-31 13:56:46.096 INFO    filelock: Lock 140054397914368 released on /home/saayed/.cache/huggingface/transformers/7588c8d398d659b230a038240cc023f67b6848117d2999f06ab625af7bfc7ec1.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b.lock


2020-12-31 13:56:47.209 INFO    filelock: Lock 140080564015360 acquired on /home/saayed/.cache/huggingface/transformers/f5316f64f9716436994a7ad76a354dc20ecb2dd74eb61d278f103a9c8b80291f.67d01b18f2079bd75eac0b2f2e7235768c7f26bd728e7a855a1c5acae01a91a8.lock


2020-12-31 13:56:47.614 INFO    filelock: Lock 140080564015360 released on /home/saayed/.cache/huggingface/transformers/f5316f64f9716436994a7ad76a354dc20ecb2dd74eb61d278f103a9c8b80291f.67d01b18f2079bd75eac0b2f2e7235768c7f26bd728e7a855a1c5acae01a91a8.lock


In [24]:
print(summarizer(article, max_length=90, min_length=20))

[{'summary_text': ' meditation r leave word u tommybahami123 kind tough sense distract attempt clear mind u jonny070389 give meditation mind go crazy tangent meditate year credit meditation make life easy place live think pass thinking thank leaver inspiration write .'}]
